## A. Tasks as specified for your team structure

**One headings for each task.**

#### 1. Load date and pre-process

In [76]:
import chardet   
import pandas as pd
import spacy
import stanza
import neuralcoref
import random
import numpy as np
from collections import defaultdict
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertModel, BertTokenizer
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from torch.nn.functional import softmax
import spacy
from collections import defaultdict
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk
import re


In [40]:
file_path = "/homes/Adam/NLP/ASM2/news_dataset.csv"

# 使用 with 语句确保文件正确关闭
with open(file_path, 'rb') as file:
    rawdata = file.read()

result = chardet.detect(rawdata)
encoding = result['encoding']

# 在尝试读取之前打印编码以确认其正确性
print(f"Detected encoding: {encoding}")

# 检查编码是否被正确检测到，然后尝试读取文件
if encoding:
    df = pd.read_csv(file_path, encoding='latin1')
else:
    print("Failed to detect encoding. Consider specifying encoding manually.")


Detected encoding: utf-8


In [41]:
sample_df = df.sample(n=100, random_state=np.random.seed(0))
sample_df.head()
# 随机抽取200个样本，加快训练速度

,id,author,date,year,month,topic,article
993,18456,Victor Mather,13/02/2017,2017,2,politics,At least six members of the Super New Englan...
859,18308,Ken Belson,8/02/2017,2017,2,sports,"HOUSTON ? There was the game on the field, ..."
298,17636,Ron Lieber,14/01/2017,2017,1,business,When Wells Fargo announced its quarterly earni...
553,17938,Dale Russakoff,29/01/2017,2017,1,politics,When Indira Islas was in third grade at Centen...
672,18070,Laurie Goodstein,30/01/2017,2017,1,crime,"Over the past decade, Christians in the United..."


In [74]:
nlp = spacy.load('en_core_web_sm')

# Download necessary NLTK datasets
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

# 假设以下是之前定义的预处理和实体提取函数的占位符
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove special characters and numbers
    text = re.sub(r'\W+', ' ', text)
    
    # Tokenize text
    tokens = word_tokenize(text)
    
    # Remove stopwords and stem the words
    filtered_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    
    return ' '.join(filtered_tokens)

# 加载spaCy模型
nlp = spacy.load('en_core_web_sm')


def extract_entities(text):
    # 使用spaCy处理文本
    doc = nlp(text)
    
    # 提取实体
    entities = []
    for ent in doc.ents:
        entities.append((ent.text, ent.label_))
    
    return entities

[nltk_data] Downloading package punkt to /home/vscode/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:

# 清洗文本列
sample_df['cleaned_article'] = sample_df['article'].apply(clean_text)

# 向量化处理过的文本
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(sample_df['cleaned_article'])
# tfidf_matrix是处理过的数值数据，可以用于后续的NLP任务


In [43]:
# 应用函数提取每篇文章的实体
sample_df['entities'] = sample_df['article'].apply(extract_entities)

doc_id_to_index = {doc_id: index for index, doc_id in enumerate(sample_df.index)}
#因为索引为随机的，所以需要一个字典来映射doc_id到索引

inverted_index = defaultdict(set)
for index, row in sample_df.iterrows():
    for entity in row['entities']:
        inverted_index[entity[0]].add(index)  # 假设entity[0]是实体的文本

In [44]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse,ner,coref', use_gpu=True)

def coref_resolve(text, nlp):
    try:
        doc = nlp(text)
        representative_mentions = []
        for sentence in doc.sentences:
            for token in sentence.tokens:
                for word in token.words:
                    if hasattr(word, 'coref_chains'):
                        for chain in word.coref_chains:
                            if chain.is_representative:
                                representative_mentions.append(word.text)
                                break
        return representative_mentions
    except IndexError:
        print("Error processing text:", text)
        return []
    

sample_df['coref_chains'] = sample_df['article'].apply(lambda x: coref_resolve(x, nlp))


2024-04-11 10:08:23 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-04-11 10:08:24 INFO: Downloaded file to /home/vscode/stanza_resources/resources.json
2024-04-11 10:08:25 INFO: Loading these models for language: en (English):
| Processor | Package                   |
-----------------------------------------
| tokenize  | combined                  |
| mwt       | combined                  |
| pos       | combined_charlm           |
| lemma     | combined_nocharlm         |
| coref     | ontonotes_electra-large   |
| depparse  | combined_charlm           |
| ner       | ontonotes-ww-multi_charlm |

2024-04-11 10:08:25 WARNING: GPU requested, but is not available!
2024-04-11 10:08:25 INFO: Using device: cpu
2024-04-11 10:08:25 INFO: Loading: tokenize
2024-04-11 10:08:25 INFO: Loading: mwt
2024-04-11 10:08:25 INFO: Loading: pos
2024-04-11

Error processing text: When Indira Islas was in third grade at Centennial Arts Academy, a public elementary school in Gainesville, Ga. she decided it was time to get serious. It was 2006, and she was in the lowest reading group in her class. She had been in that group since arriving two years earlier, speaking no English, in Gainesville, a city of 38, 000 just northeast of Atlanta?s booming outer suburbs. But being at the bottom went against everything she believed about herself. ?I wanted to be with the smart kids,? she recalls. Starting the year before, in second grade, she read every volume of the ?Magic Tree House? books in her   library, a series about two ordinary siblings who climb into their backyard treehouse and   to Pompeii, the Wild West, the ice age, feudal Japan and beyond. ?I absolutely loved them,? she says. ?It was like going on adventures all over the world. ? It was also the opposite of her own life. Indira left Mexico for the United States at age 6 with her parents 

In [45]:
def map_entities_to_corefs(entities, coref_chains):
    entity_coref_map = {}
    for entity in entities:
        entity_text = entity[0]  # Assuming entities are tuples of (text, type)
        entity_coref_map[entity_text] = []
        for coref in coref_chains:
            if entity_text in coref:
                entity_coref_map[entity_text].append(coref)
    return entity_coref_map

# Apply this function row-wise, assuming 'coref_chains' and 'entities' are available for each row
sample_df['entity_coref_map'] = sample_df.apply(lambda row: map_entities_to_corefs(row['entities'], row['coref_chains']), axis=1)


In [46]:
def search_documents(query_text):
    # 对查询进行预处理和实体识别
    cleaned_query = clean_text(query_text)
    query_vector = vectorizer.transform([cleaned_query])
    query_entities = extract_entities(query_text)
    
    # 基于实体和共指信息查找文档
    docs_based_on_content = set()
    for entity_text, _ in query_entities:
        # 使用倒排索引直接根据实体查找文档
        docs_based_on_content.update(inverted_index.get(entity_text, []))
        
        # 使用共指信息查找相关文档
        for doc_id in sample_df.index:
            coref_map = sample_df.at[doc_id, 'entity_coref_map']
            for coref_entity, mentions in coref_map.items():
                if entity_text == coref_entity or entity_text in mentions:
                    docs_based_on_content.add(doc_id)
                    break  # 如果找到匹配，就跳出循环

    # 直接使用找到的文档进行评分和排序
    if docs_based_on_content:
        docs_indices = [doc_id_to_index[doc_id] for doc_id in docs_based_on_content if doc_id in doc_id_to_index]
        docs_tfidf = tfidf_matrix[docs_indices]
        cos_similarities = cosine_similarity(query_vector, docs_tfidf).flatten()
        
        scored_docs = sorted(zip(docs_based_on_content, cos_similarities), key=lambda x: x[1], reverse=True)
        ranked_docs = [doc[0] for doc in scored_docs]
        
        return ranked_docs
    else:
        return []


In [77]:
query_text = input("Enter your query text: ")


ranked_docs = search_documents(query_text)
if ranked_docs:
    print(f"Found documents: {ranked_docs}")
else:
    print("No documents found or query format incorrect.")


Found documents: [494]


In [48]:
# 定义全局变量
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

qa_tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
qa_model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

def get_bert_embeddings(texts):
    # 使用全局变量bert_tokenizer和bert_model处理文本
    inputs = bert_tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = bert_model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :]
    return embeddings.detach().numpy()

nlp = spacy.load("en_core_web_sm")
def find_top_n_relevant_contents(question, article_content, top_n=5, window_size=5):
    # 使用spaCy进行句子分割
    doc = nlp(article_content)
    sentences = [sent.text.strip() for sent in doc.sents]  # 获取句子文本并去除首尾空白

    # 其余的逻辑与之前相同
    paragraph_embeddings = []
    for i in range(len(sentences) - window_size + 1):
        window_sentences = ' '.join(sentences[i:i+window_size])
        window_embedding = get_bert_embeddings([window_sentences])[0]  # 假设get_bert_embeddings返回numpy数组
        paragraph_embeddings.append(window_embedding)

    paragraph_embeddings = np.array(paragraph_embeddings)
    question_embedding = get_bert_embeddings([question])[0]

    # 计算相似度并找到最相关的窗口
    similarities = cosine_similarity([question_embedding], paragraph_embeddings).flatten()
    top_n_indices = np.argsort(similarities)[-top_n:][::-1]
    top_n_contents = [(' '.join(sentences[i:i+window_size]), similarities[i]) for i in top_n_indices]

    return top_n_contents


def encode_question_and_context(question, context):
    # 使用全局变量qa_tokenizer对问题和上下文进行编码
    return qa_tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt", truncation=True, max_length=512)

# def find_answer(question, context):
#     # 使用全局变量qa_tokenizer和qa_model找到问题的答案
#     inputs = encode_question_and_context(question, context)
#     input_ids = inputs["input_ids"].tolist()[0]

#     outputs = qa_model(**inputs)
#     answer_start_scores = outputs.start_logits
#     answer_end_scores = outputs.end_logits

#     answer_start = torch.argmax(answer_start_scores)
#     answer_end = torch.argmax(answer_end_scores) + 1

#     answer = qa_tokenizer.convert_tokens_to_string(qa_tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    
#     return answer


def find_answer(question, context, top_k=3):
    inputs = encode_question_and_context(question, context)
    input_ids = inputs["input_ids"].tolist()[0]

    outputs = qa_model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # 计算开始和结束分数的softmax分布
    start_probs = softmax(answer_start_scores, dim=-1)
    end_probs = softmax(answer_end_scores, dim=-1)

    # 提取排名前top_k的开始和结束位置
    start_topk = torch.topk(start_probs, top_k)
    end_topk = torch.topk(end_probs, top_k)

    top_answers = []
    for start_index, start_score in zip(start_topk.indices[0], start_topk.values[0]):
        for end_index, end_score in zip(end_topk.indices[0], end_topk.values[0]):
            # 确保结束位置在开始位置之后
            if end_index >= start_index:
                answer = qa_tokenizer.convert_tokens_to_string(qa_tokenizer.convert_ids_to_tokens(input_ids[start_index:end_index + 1]))
                score = (start_score.item() + end_score.item()) / 2  # 简单地取平均分数作为置信度
                top_answers.append((answer, score))
                break  # 只添加每个开始位置的最佳结束位置

    # 根据置信度分数降序排列答案
    top_answers = sorted(top_answers, key=lambda x: x[1], reverse=True)[:top_k]
    return top_answers



Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [55]:
# Who is the famed chronicler of the Mao era who finished writing a history of the Cultural Revolution?

while True:
    query_text = input("Enter your query text (or type 'exit' to exit): ")
    if query_text.lower() == "exit":
        print("Exiting...")
        break
    
    # 假设ranked_docs是通过某种方式得到的文档ID列表
    ranked_docs = search_documents(query_text)
    if not ranked_docs:
        print("No documents found or query format incorrect.")
        continue

    # 这里需要定义如何根据doc_id获取到实际的文档内容，例如拼接所有相关文档的内容作为context
    all_articles = ''
    # 假设sample_df是包含文档的DataFrame，并且我们已经获取了相关的doc_ids
    for doc_id in ranked_docs:
        article = sample_df.loc[doc_id, 'article']  # 假设这样可以获取到文档内容
        all_articles += article + ' '

    
    top_n_contents = find_top_n_relevant_contents(query_text, all_articles)
    relevant_context = ' '.join([content for content, score in top_n_contents])
    print(relevant_context + '\n')

    # 使用相关内容寻找答案
    answer_from_relevant_content = find_answer(query_text, relevant_context)
    print(f"Answer from relevant content: {answer_from_relevant_content}\n")

    # 直接在整个文档上寻找答案进行对比
    answer_from_all_content = find_answer(query_text, all_articles)
    print(f"Answer from all content: {answer_from_all_content}\n")

BEIJING  ? It seemed that China?s censors had finally muzzled Yang Jisheng, the famed chronicler of the Mao era. Last year, he had finished writing a widely anticipated history of the Cultural Revolution. But officials warned him against publishing it and barred him from traveling to the United States, he has said, and he stayed muted through the 50th anniversary of the start of that bloody upheaval. Now Mr. Yang has broken that silence with the publication of his history of the Cultural Revolution, ? In a sign of how Chinese politics has chilled, Mr. Yang has said little publicly about the book. ? Since the book was published, I?ve been told not to discuss it with foreign media,? he said in a brief telephone conversation. He would not say whether he had authorized ? The World Turned Upside Down? It seemed that China?s censors had finally muzzled Yang Jisheng, the famed chronicler of the Mao era. Last year, he had finished writing a widely anticipated history of the Cultural Revolution

In [65]:
# Who is the famed chronicler of the Mao era who finished writing a history of the Cultural Revolution?

while True:
    query_text = input("Enter your query text (or type 'exit' to exit): ")
    if query_text.lower() == "exit":
        print("Exiting...")
        break
    
    # 假设ranked_docs是通过某种方式得到的文档ID列表
    ranked_docs = search_documents(query_text)
    if not ranked_docs:
        print("No documents found or query format incorrect.")
        continue

    # 这里需要定义如何根据doc_id获取到实际的文档内容，例如拼接所有相关文档的内容作为context
    all_articles = ''
    # 假设sample_df是包含文档的DataFrame，并且我们已经获取了相关的doc_ids
    for doc_id in ranked_docs:
        article = sample_df.loc[doc_id, 'cleaned_article']  # 假设这样可以获取到文档内容
        all_articles += article + ' '

    
    top_n_contents = find_top_n_relevant_contents(query_text, all_articles)
    relevant_context = ' '.join([content for content, score in top_n_contents])
    print(relevant_context + '\n')

    # 使用相关内容寻找答案
    answer_from_relevant_content = find_answer(query_text, relevant_context)
    print(f"Answer from relevant cleaned_content: {answer_from_relevant_content}\n")

    # 直接在整个文档上寻找答案进行对比
    answer_from_all_content = find_answer(query_text, all_articles)
    print(f"Answer from all cleaned_content: {answer_from_all_content}\n")

leap communist utopia mr yang chronicl tombston book publish chines hong kong 2008 yang jisheng historian editori global time ardent chines newspap said last year leav impress interest histori virtual later work display strong polit tendenc last year die famin 1958 caus great leap forward mao reckless attempt leap communist utopia mr yang chronicl tombston book publish chines hong kong 2008 yang jisheng historian editori global time ardent chines newspap said last year leav impress interest histori china mr guo ad mr yang extens access archiv new book wrote tombston instead drew hundr memoir histori studi mani publish hong kong avail onlin fewer revel previou book recent scholarship mr yang emphas much worst bloodsh came later cultur revolut mao brought back militari parti apparatu brutal enforc order fair say work emin journalist mr yang book mean first histori cultur revolut sever publish offici auspic mainland china earlier decad chines govern tri confront ordeal era recent year par

In [94]:
def auto_answer_question(query_text, use_relevant_content=True):
    # 假设ranked_docs是通过某种方式得到的文档ID列表
    ranked_docs = search_documents(query_text)
    if not ranked_docs:
        return "No documents found or query format incorrect."

    # 获取所有相关文档的内容
    all_articles = ''
    for doc_id in ranked_docs:
        # 假设可以通过doc_id获取到文档内容
        article = sample_df.loc[doc_id, 'article']
        all_articles += article + ' '

    # 根据use_relevant_content变量决定使用哪种内容寻找答案
    if use_relevant_content:
        # 找到与问题最相关的内容片段
        top_n_contents = find_top_n_relevant_contents(query_text, all_articles, top_n=3)
        relevant_context = ' '.join([content[0] for content in top_n_contents])
        context_to_use = relevant_context
    else:
        # 使用全部内容
        context_to_use = all_articles

    # 使用选择的内容寻找答案
    answer = find_answer(query_text, context_to_use)
    return answer


In [99]:
def compute_f1(predicted_answers, truths):
    f1_scores = []
    for predicted in predicted_answers:  # 不需要解包
        pred_tokens = predicted.split()
        truth_tokens = set([answer for truth in truths for answer in truth.split()])
        common = set(pred_tokens) & truth_tokens
        if not common:
            f1_scores.append(0)
            continue
        precision = len(common) / len(pred_tokens)
        recall = len(common) / len(truth_tokens)
        f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
        f1_scores.append(f1)
    return np.mean(f1_scores) if f1_scores else 0


def compute_mrr(predicted_article_ids, truth_article_ids):
    rank = 0
    for i, predicted_id in enumerate(predicted_article_ids, start=1):
        if predicted_id in truth_article_ids:
            rank = 1 / i
            break
    return rank


def compute_map(predicted_article_ids, truth_article_ids):
    avg_precisions = []
    for i, predicted_id in enumerate(predicted_article_ids, start=1):
        if predicted_id in truth_article_ids:
            relevant_count = sum(pred_id in truth_article_ids for pred_id in predicted_article_ids[:i])
            precision_at_i = relevant_count / i
            avg_precisions.append(precision_at_i)
    return np.mean(avg_precisions) if avg_precisions else 0




def lcs_length(x, y):
    """计算两个序列的最长公共子序列（LCS）的长度"""
    if not x or not y:
        return 0
    dp = [[0] * (len(y) + 1) for _ in range(len(x) + 1)]
    for i in range(1, len(x) + 1):
        for j in range(1, len(y) + 1):
            if x[i - 1] == y[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
            else:
                dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])
    return dp[-1][-1]

def compute_rouge_l_multi(predicted_answers, truths):
    def compute_rouge_l(predicted, truth):
        """计算单个预测答案与单个真实答案之间的ROUGE-L分数。"""
        lcs = lcs_length(predicted, truth)
        if lcs == 0:
            return 0
        precision = lcs / len(predicted)
        recall = lcs / len(truth)
        f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
        return f1
    
    rouge_l_scores = []
    for predicted in predicted_answers:  # 直接处理每个预测答案文本
        # 计算当前预测答案与每个真实答案之间的ROUGE-L分数，并取最大值
        scores_for_this_answer = [compute_rouge_l(predicted, truth) for truth in truths]
        rouge_l_scores.append(max(scores_for_this_answer))
    
    return np.mean(rouge_l_scores) if rouge_l_scores else 0




In [90]:
test_data = [
    {
        "question": "Who is the famed chronicler of the Mao era who finished writing a history of the Cultural Revolution?",
        "answers": ["Yang Jisheng"]
    },
    {
        "question": "What is the title of Yang Jisheng's book about the Cultural Revolution?",
        "answers": ["The World Turned Upside Down"]
    },
    {
        "question": "In which year did Xi Jinping come to power in China?",
        "answers": ["2012"]
    },
    {
        "question": "What was Yang Jisheng's role during the early phase of the Cultural Revolution?",
        "answers": ["He was a university student in Beijing who immersed himself in the early phase."]
    },
    {
        "question": "What is described as 'historical nihilism' by the Chinese government?",
        "answers": ["Delving into events like the Cultural Revolution, described as subversive to corrode the party’s authority."]
    },
    {
        "question": "List two challenges Yang Jisheng faced while writing his historical works.",
        "answers": ["He was warned against publishing his book and barred from traveling to the United States; he was told not to discuss the book with foreign media."]
    },
    {
        "question": "Why did Yang Jisheng decide to write a book about the Cultural Revolution?",
        "answers": ["To expose lies and restore the truth."]
    },
    {
        "question": "Why was Yang Jisheng advised not to discuss his book with foreign media after its publication?",
        "answers": ["The article implies there's political pressure, but does not provide a specific reason."]
    },
    {
        "question": "How many people were killed during the Cultural Revolution, and what does this reflect?",
        "answers": ["Perhaps a million or more people were killed, reflecting the extreme turmoil and bloodshed of that period."]
    },
    {
        "question": "What is the name of Yang Jisheng's other book about the famine caused by the Great Leap Forward?",
        "answers": ["Tombstone."]
    },
    {
        "question": "Which publishing company plans to publish an abridged English version of 'The World Turned Upside Down' in 2019?",
        "answers": ["Farrar, Straus and Giroux."]
    },
    {
        "question": "What year did the Cultural Revolution begin?",
        "answers": ["1966"]
    },
    {
        "question": "How did Yang Jisheng participate in the Cultural Revolution?",
        "answers": ["He threw himself into the early phase when Mao unleashed student radicals."]
    },
    {
        "question": "What does Yang Jisheng aim to achieve with his writing, as mentioned in his book on the Cultural Revolution?",
        "answers": ["To expose lies and restore the truth."]
    },    {
        "question": 'Why has Hong Kong been a significant location for the publication of books like The World Turned Upside Down?',
        "answers": ["Because it offers much greater freedom than is found in mainland China, allowing for the publication of books banned in the mainland."]
    },
]


In [108]:
questions_answers = {
    "fact_based": [
        {
            "question": "What Netflix series is a reboot of a Norman Lear sitcom that discusses class divide?",
            "answer": "One Day at a Time",
            "article_id": 77
        },
        {
            "question": "Who was the American ambassador to the United Nations who issued a warning to allies and rivals in her first remarks?",
            "answer": "Nikki R. Haley",
            "article_id": 614
        },
        {
            "question": "In which year did Xi Jinping come to power in China?",
            "answer": ["2012"],
            "article_id": 494
        },
        {
            "question": "What is the title of Yang Jisheng's book about the Cultural Revolution?",
            "answer": ["The World Turned Upside Down"],
            "article_id": 494
        },
        {
            "question": "What year did the Cultural Revolution begin?",
            "answer": ["1966"],
            "article_id": 494
        },
        {
            "question": "What is the name of Yang Jisheng's other book about the famine caused by the Great Leap Forward?",
            "answer": ["Tombstone"],
            "article_id": 494
        },
        {
            "question": "Which publishing company plans to publish an abridged English version of 'The World Turned Upside Down' in 2019?",
            "answer": ["Farrar, Straus and Giroux"],
            "article_id": 494
        }
    ],
    "explanation_based": [
        {
            "question": "How does 'One Day at a Time' reflect the issue of class divide in its narrative?",
            "answer": "The show reflects the class divide by showcasing a family that is closer to the lower side of the economic spectrum, discussing real-life economic struggles within a sitcom format.",
            "article_id": 77
        },
        {
            "question": "What was the main message Nikki R. Haley conveyed in her first remarks at the United Nations?",
            "answer": "The main message was that the Trump administration would hold to account those who do not back the United States, signaling a change in the way the U.S. interacts with the UN.",
            "article_id": 614
        },
        {
            "question": "Who is the famed chronicler of the Mao era who finished writing a history of the Cultural Revolution?",
            "answer": ["Yang Jisheng"],
            "article_id": 494
        },
        {
            "question": "What was Yang Jisheng's role during the early phase of the Cultural Revolution?",
            "answer": ["He was a university student in Beijing who immersed himself in the early phase."],
            "article_id": 494
        },
        {
            "question": "What is described as 'historical nihilism' by the Chinese government?",
            "answer": ["Delving into events like the Cultural Revolution, described as subversive to corrode the party’s authority."],
            "article_id": 494
        },
        {
            "question": "Why did Yang Jisheng decide to write a book about the Cultural Revolution?",
            "answer": ["To expose lies and restore the truth."],
            "article_id": 494
        },
        {
            "question": "Why was Yang Jisheng advised not to discuss his book with foreign media after its publication?",
            "answer": ["The article implies there's political pressure, but does not provide a specific reason."],
            "article_id": 494
        },
        {
            "question": "How many people were killed during the Cultural Revolution, and what does this reflect?",
            "answer": ["Perhaps a million or more people were killed, reflecting the extreme turmoil and bloodshed of that period."],
            "article_id": 494
        },
        {
            "question": "How did Yang Jisheng participate in the Cultural Revolution?",
            "answer": ["He threw himself into the early phase when Mao unleashed student radicals."],
            "article_id": 494
        },
        {
            "question": "What does Yang Jisheng aim to achieve with his writing, as mentioned in his book on the Cultural Revolution?",
            "answer": ["To expose lies and restore the truth."],
            "article_id": 494
        },
        {
            "question": "Why has Hong Kong been a significant location for the publication of books like 'The World Turned Upside Down'?",
            "answer": ["Because it offers much greater freedom than is found in mainland China, allowing for the publication of books banned in the mainland."],
            "article_id": 494
        }
    ],
    "list_based": [
        {
            "question": "Who are the main characters in the Netflix reboot of 'One Day at a Time'?",
            "answer": ["Penelope", "Alex", "Lydia", "Elena"],
            "article_id": 77
        },
        {
            "question": "What are the implications of the Trump administration's stance towards the United Nations as expressed by Nikki R. Haley?",
            "answer": ["Holding to account those who do not support the U.S.", "A potential reduction in U.S. funding for the UN", "A focus on showing strength and value in U.N. participation"],
            "article_id": 614
        },
        {
            "question": "List two challenges Yang Jisheng faced while writing his historical works.",
            "answer": ["He was warned against publishing his book and barred from traveling to the United States; he was told not to discuss the book with foreign media."],
            "article_id": 494
        }
    ]
}


In [103]:
def run_evaluation(test_data, qa_system_func, use_relevant_content=True):
    f1_scores, mrr_scores, map_scores, rouge_l_scores = [], [], [], []

    for item in test_data:
        question, true_answers , true_article_ids= item["question"], item["answers"],item["article_ids"]
        
        predicted_answers_with_scores = qa_system_func(question, use_relevant_content)
        
        predicted_answers = [ans[0] for ans in predicted_answers_with_scores]

        f1 = compute_f1(predicted_answers, true_answers)
        mrr = compute_mrr(predicted_answers, true_answers)
        map_score = compute_map(predicted_answers, true_answers)
        rouge_l = compute_rouge_l_multi(predicted_answers, true_answers)

        f1_scores.append(f1)
        mrr_scores.append(mrr)
        map_scores.append(map_score)
        rouge_l_scores.append(rouge_l)
    
    print(f"Average F1 Score: {np.mean(f1_scores)}")
    print(f"Average MRR: {np.mean(mrr_scores)}")
    print(f"Average MAP: {np.mean(map_scores)}")
    print(f"Average ROUGE-L: {np.mean(rouge_l_scores)}")


In [106]:
print(test_data[0])

KeyError: 0

In [109]:
import numpy as np

def compute_f1_single(predicted, truth):
    pred_tokens = set(predicted.lower().split())
    truth_tokens = set(truth.lower().split())
    common_tokens = pred_tokens.intersection(truth_tokens)
    if not common_tokens:
        return 0.0
    precision = len(common_tokens) / len(pred_tokens)
    recall = len(common_tokens) / len(truth_tokens)
    if precision + recall == 0:
        return 0.0
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

def qa_system_func(question, article_id, use_relevant_content):
    # 这里是一个假设的函数实现，实际应用中应替换为你的问答系统逻辑
    # 假设它总是返回问题中的最后一个词作为答案（仅用于演示）
    return question.split()[-1]

def run_evaluation(test_data, qa_system_func, use_relevant_content=True):
    for base_type, questions in test_data.items():
        f1_scores = []
        
        for item in questions:
            question = item["question"]
            true_answer = item["answer"]  # Assuming a single answer for simplicity
            article_id = item["article_id"]
            
            predicted_answer = qa_system_func(question, article_id, use_relevant_content)
            
            f1 = compute_f1_single(predicted_answer, true_answer)
            f1_scores.append(f1)
        
        average_f1 = np.mean(f1_scores) if f1_scores else 0.0
        print(f"Average F1 Score for {base_type}: {average_f1:.3f}")

# 假设你有一个questions_answers字典，已经定义了按照上面格式
# 
run_evaluation(questions_answers, qa_system_func)


AttributeError: 'list' object has no attribute 'lower'

In [101]:

print('Rustl with whole content')

run_evaluation(test_data, auto_answer_question,use_relevant_content=False)

Rustl with whole content
Average F1 Score: 0.15787558351890046
Average MRR: 0.06666666666666667
Average MAP: 0.06666666666666667
Average ROUGE-L: 0.3063513571999547


In [75]:
print('Rustl with relevant content')

run_evaluation(test_data, auto_answer_question)

Rustl with relevant content
Average F1 Score: 0.0743491972377193
Average MRR: 0.0
Average MAP: 0.08080357841639846
Average ROUGE-L: 0.2945520255253334


## B. References

## C. Appendix